# Variational autoencoder
* Original paper: Auto-Encoding Variational Bayes (https://arxiv.org/pdf/1312.6114.pdf)

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

from tqdm import tqdm

batch_size = 128
epochs = 10
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [2]:
# MNIST
root = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda x: x.view(-1))])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=True, transform=transform, download=True),
    shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root=root, train=False, transform=transform),
    shuffle=False, **kwargs)

In [3]:
from pixyz.utils import print_latex

## Define probability distributions
Prior: $p(z) = \cal N(z; \mu=0, \sigma^2=1)$  
Generator: $p_{\theta}(x|z) = \cal B(x; \lambda = g(z))$  
Inference: $q_{\phi}(z|x) = \cal N(z; \mu=f_\mu(x), \sigma^2=f_{\sigma^2}(x))$

In [4]:
from pixyz.distributions import Normal, Bernoulli

x_dim = 784
z_dim = 64


# inference model q(z|x)
class Inference(Normal):
    """
    parameterizes q(z | x)
    infered z follows a Gaussian distribution with mean 'loc', variance 'scale'
    z ~ N(loc, scale)
    """
    def __init__(self):
        super(Inference, self).__init__(var=["z"], cond_var=["x"], name="q")

        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)

    def forward(self, x):
        """
        given the observation x,
        return the mean and variance of the Gaussian distritbution
        """
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}

    
# generative model p(x|z)    
class Generator(Bernoulli):
    """
    parameterizes the bernoulli(for MNIST) observation likelihood p(x | z)
    """
    def __init__(self):
        super(Generator, self).__init__(var=["x"], cond_var=["z"], name="p")

        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)

    def forward(self, z):
        """
        given the latent variable z,
        return the probability of Bernoulli distribution
        """
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {"probs": torch.sigmoid(self.fc3(h))}
    
p = Generator().to(device)
q = Inference().to(device)

#  prior p(z)
# z ~ N(0, 1)
prior = Normal(loc=torch.tensor(0.), scale=torch.tensor(1.),
               var=["z"], features_shape=[z_dim], name="p_{prior}").to(device)

In [5]:
print(prior)
print_latex(prior)

Distribution:
  p_{prior}(z)
Network architecture:
  Normal(
    name=p_{prior}, distribution_name=Normal,
    var=['z'], cond_var=[], input_var=[], features_shape=torch.Size([64])
    (loc): torch.Size([1, 64])
    (scale): torch.Size([1, 64])
  )


<IPython.core.display.Math object>

In [6]:
print(p)
print_latex(p)

Distribution:
  p(x|z)
Network architecture:
  Generator(
    name=p, distribution_name=Bernoulli,
    var=['x'], cond_var=['z'], input_var=['z'], features_shape=torch.Size([])
    (fc1): Linear(in_features=64, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (fc3): Linear(in_features=512, out_features=784, bias=True)
  )


<IPython.core.display.Math object>

In [7]:
print(q)
print_latex(q)

Distribution:
  q(z|x)
Network architecture:
  Inference(
    name=q, distribution_name=Normal,
    var=['z'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (fc1): Linear(in_features=784, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=512, bias=True)
    (fc31): Linear(in_features=512, out_features=64, bias=True)
    (fc32): Linear(in_features=512, out_features=64, bias=True)
  )


<IPython.core.display.Math object>

## Define Loss function
Loss function:

$\frac{1}{N} \sum_{i=1}^{N}\left[K L\left(q\left(z | x^{(i)}\right) \| p_{prior}(z)\right)-\mathbb{E}_{q\left(z | x^{(i)}\right)}\left[\log p\left(x^{(i)} | z\right)\right]\right]$

In [8]:
from pixyz.losses import LogProb, KullbackLeibler, Expectation as E

loss = (KullbackLeibler(q, prior) - E(q, LogProb(p))).mean()
print_latex(loss)

<IPython.core.display.Math object>

## Define VAE model using Model Class
- https://docs.pixyz.io/en/latest/models.html#model

In [9]:
from pixyz.models import Model

model = Model(loss=loss, distributions=[p, q],
              optimizer=optim.Adam, optimizer_params={"lr":1e-3})
print(model)
print_latex(model)

Distributions (for training): 
  p(x|z), q(z|x) 
Loss function: 
  mean \left(D_{KL} \left[q(z|x)||p_{prior}(z) \right] - \mathbb{E}_{q(z|x)} \left[\log p(x|z) \right] \right) 
Optimizer: 
  Adam (
  Parameter Group 0
      amsgrad: False
      betas: (0.9, 0.999)
      eps: 1e-08
      lr: 0.001
      weight_decay: 0
  )


<IPython.core.display.Math object>

## Define Train and Test loop using model

In [10]:
def train(epoch):
    train_loss = 0
    for x, _ in tqdm(train_loader):
        x = x.to(device)
        loss = model.train({"x": x})
        train_loss += loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [11]:
def test(epoch):
    test_loss = 0
    for x, _ in test_loader:
        x = x.to(device)
        loss = model.test({"x": x})
        test_loss += loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

## Reconstruct image and generate image

In [12]:
def plot_reconstrunction(x):
    """
    reconstruct image given input observation x
    """
    with torch.no_grad():
        # infer and sampling z using inference model q `.sample()` method
        z = q.sample({"x": x}, return_all=False)
        
        # reconstruct image from inferred latent variable z using Generator model p `.sample_mean()` method
        recon_batch = p.sample_mean(z).view(-1, 1, 28, 28)
        
        # concatenate original image and reconstructed image for comparison
        comparison = torch.cat([x.view(-1, 1, 28, 28), recon_batch]).cpu()
        return comparison


def plot_image_from_latent(z_sample):
    """
    generate new image given latent variable z
    """
    with torch.no_grad():
        # generate image from latent variable z using Generator model p `.sample_mean()` method
        sample = p.sample_mean({"z": z_sample}).view(-1, 1, 28, 28).cpu()
        return sample

In [13]:
# for visualising in TensorBoard
import pixyz
import datetime

dt_now = datetime.datetime.now()
exp_time = dt_now.strftime('%Y%m%d_%H:%M:%S')
v = pixyz.__version__
nb_name = 'vae'
writer = SummaryWriter("runs/" + v + "." + nb_name + exp_time)

# fix latent variable z for watching generative model improvement 
z_sample = 0.5 * torch.randn(64, z_dim).to(device)

# set-aside observation for watching generative model improvement 
_x, _ = iter(test_loader).next()
_x = _x.to(device)

for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)
    
    recon = plot_reconstrunction(_x[:8])
    sample = plot_image_from_latent(z_sample)

    writer.add_scalar('train_loss', train_loss.item(), epoch)
    writer.add_scalar('test_loss', test_loss.item(), epoch)      
    
    writer.add_images('Image_from_latent', sample, epoch)
    writer.add_images('Image_reconstrunction', recon, epoch)
    
writer.close()

100%|██████████| 469/469 [00:08<00:00, 52.78it/s]

Epoch: 1 Train loss: 179.0260



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 142.2970


100%|██████████| 469/469 [00:09<00:00, 51.62it/s]

...

  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 104.3698


100%|██████████| 469/469 [00:09<00:00, 47.96it/s]

Epoch: 10 Train loss: 103.3093


Test loss: 104.0028
